In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")
import pickle
import numpy as np

In [ ]:
dataset1 = pd.read_csv('/content/student-scores.csv')
dataset = dataset1.copy()
dataset.head()


,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,1,Paul,Casey,paul.casey.1@gslingacademy.com,male,False,3,False,27,Lawyer,73,81,93,97,63,80,87
1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,female,False,2,False,47,Doctor,90,86,96,100,90,88,90
2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,female,False,9,True,13,Government Officer,81,97,95,96,65,77,94
3,4,Tara,Clark,tara.clark.4@gslingacademy.com,female,False,5,False,3,Artist,71,74,88,80,89,63,86
4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,male,False,5,False,10,Unknown,84,77,65,65,80,74,76


Drop Irrelevant columns


In [ ]:
dataset.drop(columns=['id', 'first_name', 'last_name', 'email'], axis=1, inplace=True)

In [ ]:
dataset.columns

Index(['gender', 'part_time_job', 'absence_days', 'extracurricular_activities',
       'weekly_self_study_hours', 'career_aspiration', 'math_score',
       'history_score', 'physics_score', 'chemistry_score', 'biology_score',
       'english_score', 'geography_score'],
      dtype='object')

In [ ]:
dataset

,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,male,False,3,False,27,Lawyer,73,81,93,97,63,80,87
1,female,False,2,False,47,Doctor,90,86,96,100,90,88,90
2,female,False,9,True,13,Government Officer,81,97,95,96,65,77,94
3,female,False,5,False,3,Artist,71,74,88,80,89,63,86
4,male,False,5,False,10,Unknown,84,77,65,65,80,74,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,male,False,2,False,30,Construction Engineer,83,77,84,73,75,84,82
1996,male,False,2,False,20,Software Engineer,89,65,73,80,87,67,73
1997,female,False,5,False,14,Software Engineer,97,85,63,93,68,94,78
1998,female,True,10,True,5,Business Owner,51,96,72,89,95,88,75


Create new features from all score (Feature Engineering)

In [ ]:
dataset['total_score'] = dataset['math_score'] + dataset['history_score'] + dataset['physics_score'] + dataset['chemistry_score'] + dataset['biology_score'] + dataset['english_score'] +  dataset['geography_score']
dataset['average_score'] = dataset['total_score'] / 7
dataset.head()

,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score,total_score,average_score
0,male,False,3,False,27,Lawyer,73,81,93,97,63,80,87,574,82.000000
1,female,False,2,False,47,Doctor,90,86,96,100,90,88,90,640,91.428571
2,female,False,9,True,13,Government Officer,81,97,95,96,65,77,94,605,86.428571
3,female,False,5,False,3,Artist,71,74,88,80,89,63,86,551,78.714286
4,male,False,5,False,10,Unknown,84,77,65,65,80,74,76,521,74.428571


In [ ]:
dataset['career_aspiration'].unique()

array(['Lawyer', 'Doctor', 'Government Officer', 'Artist', 'Unknown',
       'Software Engineer', 'Teacher', 'Business Owner', 'Scientist',
       'Banker', 'Writer', 'Accountant', 'Designer',
       'Construction Engineer', 'Game Developer', 'Stock Investor',
       'Real Estate Developer'], dtype=object)

In [ ]:
len(dataset['career_aspiration'].unique())

17

Encoding Categorical Columns,convert categories into numbers

In [ ]:
gender_map = {'male': 0, 'female': 1}
part_time_job_map = {False: 0, True: 1}
extracurricular_activities_map = {False: 0, True: 1}
career_aspiration_map = {
    'Lawyer': 0, 'Doctor': 1, 'Government Officer': 2, 'Artist': 3, 'Unknown': 4,
    'Software Engineer': 5, 'Teacher': 6, 'Business Owner':7, 'Scientist': 8,
    'Banker':9, 'Writer': 10, 'Accountant': 11, 'Designer': 12,
    'Construction Engineer': 13, 'Game Developer': 14, 'Stock Investor': 15,
    'Real Estate Developer': 16
}

# Apply to the DataFrame
dataset['gender'] = dataset['gender'].map(gender_map)
dataset['part_time_job'] = dataset['part_time_job'].map(part_time_job_map)
dataset['extracurricular_activities'] = dataset['extracurricular_activities'].map(extracurricular_activities_map)
dataset['career_aspiration'] = dataset['career_aspiration'].map(career_aspiration_map)


In [ ]:
dataset

,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score,total_score,average_score
0,0,0,3,0,27,0,73,81,93,97,63,80,87,574,82.000000
1,1,0,2,0,47,1,90,86,96,100,90,88,90,640,91.428571
2,1,0,9,1,13,2,81,97,95,96,65,77,94,605,86.428571
3,1,0,5,0,3,3,71,74,88,80,89,63,86,551,78.714286
4,0,0,5,0,10,4,84,77,65,65,80,74,76,521,74.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,0,2,0,30,13,83,77,84,73,75,84,82,558,79.714286
1996,0,0,2,0,20,5,89,65,73,80,87,67,73,534,76.285714
1997,1,0,5,0,14,5,97,85,63,93,68,94,78,578,82.571429
1998,1,1,10,1,5,7,51,96,72,89,95,88,75,566,80.857143


In [ ]:
dataset.shape

(2000, 15)

Balance Dataset

In [ ]:
dataset['career_aspiration'].value_counts()

career_aspiration
5     315
7     309
4     223
9     169
0     138
11    126
1     119
16     83
15     73
13     68
3      67
14     63
2      61
6      59
12     56
8      39
10     32
Name: count, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE
# Create SMOTE object
smote = SMOTE(random_state=42)

# Separate features and target variable
X = dataset.drop('career_aspiration', axis=1)
y = dataset['career_aspiration']

# Apply SMOTE to the data
X_resampled, y_resampled = smote.fit_resample(X,y)

In [ ]:
y_resampled.value_counts()

career_aspiration
0     315
9     315
15    315
14    315
13    315
12    315
11    315
10    315
8     315
1     315
7     315
6     315
5     315
4     315
3     315
2     315
16    315
Name: count, dtype: int64

In [ ]:
y_resampled.shape

(5355,)

Train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4284, 14), (4284,), (1071, 14), (1071,))

Feature Scalling (give all values equal importance, priority)

In [ ]:
X_train

,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score,total_score,average_score
2369,1,0,3,0,38,96,93,93,90,87,87,90,639,91.386647
3955,1,0,3,0,29,95,77,93,85,47,69,74,542,77.464726
3785,1,0,1,0,16,71,99,98,95,91,95,95,648,92.576635
120,1,1,0,0,13,66,90,83,80,76,90,91,576,82.285714
3699,1,0,3,0,21,68,77,82,92,92,90,71,574,82.071885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3092,1,0,2,0,11,65,73,92,61,93,92,86,565,80.807391
3772,0,0,2,0,21,71,72,84,89,74,93,65,551,78.854845
5191,0,0,5,0,3,87,77,64,83,83,93,79,569,81.405332
5226,1,0,3,0,4,75,91,67,86,79,72,88,562,80.361061


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
X_train_scaled

array([[ 1.28299565, -0.25678162, -0.00818145, ...,  0.83256957,
         1.69887402,  1.7074715 ],
       [ 1.28299565, -0.25678162, -0.00818145, ..., -0.6591611 ,
        -0.80753861, -0.80832155],
       [ 1.28299565, -0.25678162, -0.93053284, ...,  1.29873541,
         1.93142777,  1.92251092],
       ...,
       [-0.77942587, -0.25678162,  0.91416993, ..., -0.19299527,
        -0.10987736, -0.09622524],
       [ 1.28299565, -0.25678162, -0.00818145, ...,  0.64610324,
        -0.2907525 , -0.28493267],
       [-0.77942587, -0.25678162,  0.45299424, ..., -0.28622844,
        -0.93673514, -0.94393154]])

Models Training(Multiple Models)

In [ ]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Support Vector Classifier": SVC(),
    "Random Forest Classifier": RandomForestClassifier(),
    "K Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree Classifier": DecisionTreeClassifier(),
    "Gaussian Naive Bayes": GaussianNB(),
    "Adaboost Classifier": AdaBoostClassifier(),
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "XGBoost Classifier": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    }

# Train and evaluate each model
for name, model in models.items():
  print("="*50)
  print("Model:", name)
  # Train the model
  model.fit(X_train_scaled, y_train)


  # Predict on set
  y_pred = model.predict(X_test_scaled)

  # Calculate metrics
  accuracy = accuracy_score(y_test, y_pred)
  classification_rep = classification_report(y_test, y_pred)
  conf_matrix = confusion_matrix(y_test, y_pred)


  print("Accuracy:", accuracy)
  print("Classification Report:\n", classification_rep)
  print("Confusion Matrix:\n", conf_matrix)


Model: Logistic Regression
Accuracy: 0.48552754435107376
Classification Report:
               precision    recall  f1-score   support

           0       0.45      0.54      0.49        68
           1       0.48      0.62      0.55        72
           2       0.41      0.44      0.42        57
           3       0.51      0.55      0.53        58
           4       0.33      0.17      0.22        66
           5       0.32      0.32      0.32        76
           6       0.58      0.92      0.71        71
           7       0.83      0.90      0.87        61
           8       0.41      0.45      0.43        53
           9       0.29      0.10      0.15        61
          10       0.59      0.71      0.65        63
          11       0.44      0.45      0.45        53
          12       0.31      0.16      0.21        68
          13       0.38      0.49      0.43        55
          14       0.60      0.91      0.73        57
          15       0.37      0.24      0.29        63


Model Selection (Random Forest)

In [ ]:
model = RandomForestClassifier()

model.fit(X_train_scaled, y_train)
# Predict on test set
y_pred = model.predict(X_test_scaled)

# Calculate metrics
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.8272642390289449
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.85      0.82        68
           1       0.83      0.94      0.88        72
           2       0.81      0.91      0.86        57
           3       0.87      0.91      0.89        58
           4       0.88      0.42      0.57        66
           5       0.55      0.42      0.48        76
           6       0.92      0.97      0.95        71
           7       0.95      0.93      0.94        61
           8       0.84      1.00      0.91        53
           9       0.61      0.70      0.66        61
          10       0.98      0.97      0.98        63
          11       0.75      0.72      0.73        53
          12       0.91      0.93      0.92        68
          13       0.75      0.91      0.82        55
          14       0.92      0.98      0.95        57
          15       0.78      0.71      0.74        63
          16       0.94     

In [ ]:
# Train both models on the same data
rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)

xgb_model = XGBClassifier()
xgb_model.fit(X_train_scaled, y_train)

# Predict on the test data
rf_predictions = rf_model.predict(X_test_scaled)
xgb_predictions = xgb_model.predict(X_test_scaled)

# Average the predictions
combined_predictions = (rf_predictions + xgb_predictions) / 2


# Evaluate the combined predictions
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.8272642390289449
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.85      0.82        68
           1       0.83      0.94      0.88        72
           2       0.81      0.91      0.86        57
           3       0.87      0.91      0.89        58
           4       0.88      0.42      0.57        66
           5       0.55      0.42      0.48        76
           6       0.92      0.97      0.95        71
           7       0.95      0.93      0.94        61
           8       0.84      1.00      0.91        53
           9       0.61      0.70      0.66        61
          10       0.98      0.97      0.98        63
          11       0.75      0.72      0.73        53
          12       0.91      0.93      0.92        68
          13       0.75      0.91      0.82        55
          14       0.92      0.98      0.95        57
          15       0.78      0.71      0.74        63
          16       0.94     

Single Input Predictions

In [ ]:
X_test_scaled[10].reshape(1, -1)

array([[ 1.28299565, -0.25678162,  1.37534562, -0.33832543, -0.20860229,
        -0.29448353,  0.84117249,  1.40056418,  0.52088807,  0.49520339,
         0.98111738,  1.67166808,  1.69887402,  1.69331945]])

In [ ]:
print("Predicted Label :", model.predict(X_test_scaled[10].reshape(1, -1))[0])
print("Actual Label :", y_test.iloc[10])

Predicted Label : 12
Actual Label : 12


In [ ]:
print("Predicted Label :", model.predict(X_test_scaled[4].reshape(1, -1))[0])
print("Actual Label :", y_test.iloc[4])

Predicted Label : 4
Actual Label : 4


In [ ]:
print("Predicted Label :", model.predict(X_test_scaled[55].reshape(1, -1))[0])
print("Actual Label :", y_test.iloc[55])

Predicted Label : 10
Actual Label : 10


Saving and Loading Files

In [ ]:
# SAVE FILES
pickle.dump(scaler, open("scaler.pkl",'wb'))
pickle.dump(model, open("model.pkl", 'wb'))

In [ ]:
# Load the scaler, lebel encoder and model
scaler = pickle.load(open('/content/scaler.pkl', 'rb'))
model = pickle.load(open('/content/model.pkl', 'rb'))


Recommendation System

In [ ]:
class_names = ['Lawyer', 'Doctor', 'Government Officer', 'Artist', 'Unknown',
               'Software Engineer', 'Teacher', 'Business Owner', 'Scientist',
               'Banker', 'Writer', 'Accountant', 'Designer',
               'Construction Engineer', 'Game Developer', 'Stock Investor',
               'Real Estate Developer']

def Recommendations(gender, part_time_job, absence_days, extracurricular_activities,
                    weekly_self_study_hours, math_score, history_score, physics_score,
                    chemistry_score, biology_score, english_score, geography_score,
                    total_score, average_score):

  # Encode categorical variables
    gender_encoded = 1 if gender.lower() == 'female' else 0
    part_time_job_encoded = 1 if part_time_job else 0
    extracurricular_activities_encoded = 1 if extracurricular_activities else 0


    # Create feature array
    feature_array = np.array([[gender_encoded, part_time_job_encoded, absence_days, extracurricular_activities_encoded,
                               weekly_self_study_hours, math_score, history_score, physics_score,
                               chemistry_score, biology_score, english_score, geography_score, total_score,
                               average_score]])

    # Scale features
    scaled_features = scaler.transform(feature_array)


    # Predict using the model
    probabilities = model.predict_proba(scaled_features)


    # Get top five predicted classes along with their probabilities
    top_classes_idx = np.argsort(-probabilities[0])[:5]
    top_classes_names_probs = [(class_names[idx], probabilities[0][idx]) for idx in top_classes_idx]



    return top_classes_names_probs



In [ ]:
# Example usage 1
final_recommendations = Recommendations(gender='female',
                                        part_time_job=False,
                                        absence_days=2,
                                        extracurricular_activities=False,
                                        weekly_self_study_hours=7,
                                        math_score=65,
                                        history_score=60,
                                        physics_score=97,
                                        chemistry_score=94,
                                        biology_score=81,
                                        english_score=81,
                                        geography_score=66,
                                        total_score=534,
                                        average_score=76.285714
                                        )
print("Top recommended studies with probabilities:")
print("="*50)
for class_name, probability in final_recommendations:
  print(f"{class_name} with probabiity {probability}")

Top recommended studies with probabilities:
Teacher with probabiity 0.59
Unknown with probabiity 0.27
Government Officer with probabiity 0.06
Real Estate Developer with probabiity 0.02
Designer with probabiity 0.02


In [ ]:
# Example usage 2
final_recommendations = Recommendations(gender='female',
                                        part_time_job=False,
                                        absence_days=2,
                                        extracurricular_activities=False,
                                        weekly_self_study_hours=4,
                                        math_score=87,
                                        history_score=73,
                                        physics_score=98,
                                        chemistry_score=91,
                                        biology_score=79,
                                        english_score=60,
                                        geography_score=77,
                                        total_score=583,
                                        average_score=83.285714
                                        )
print("Top recommended studies with probabilities:")
print("="*50)
for class_name, probability in final_recommendations:
  print(f"{class_name} with probabiity {probability}")

Top recommended studies with probabilities:
Artist with probabiity 0.63
Game Developer with probabiity 0.16
Real Estate Developer with probabiity 0.09
Doctor with probabiity 0.02
Unknown with probabiity 0.02


In [168]:
# sklearn version in pycharm production
import sklearn
print(sklearn.__version__)
# in pycharm env install
# pip install scikit-learn==1.2.2

1.2.2
